<a href="https://colab.research.google.com/github/LennartKeller/TextklassifikationsProjekt2019/blob/master/DistilBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall --yes tensorflow
!pip install tensorflow>=2.0.0

!pip install transformers
!pip install ktrain

Uninstalling tensorflow-1.15.0:
  Successfully uninstalled tensorflow-1.15.0
ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.2 which is incompatible.
     |████████████████████████████████| 450kB 5.1MB/s 
     |████████████████████████████████| 1.0MB 52.1MB/s 
     |████████████████████████████████| 870kB 58.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884629 sha256=f31c3c579530605e5103ba5456799b455809738925c206f5bb35b8f50673e33a
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
     |████████████████████████████████| 102kB 4.4MB/s 
     |████████████████████████████████| 6.7MB 13.1MB/s 
     |████████████████████████████████| 1.0MB 51.1MB/s 
     |████████████████████████████████| 245kB 66.8MB/s 
     

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd

In [0]:
df = pd.read_csv('/content/gdrive/My Drive/full_dataset.csv').dropna()

In [0]:
MODEL_NAME = 'bert-base-german-cased' # this modell does not work well (60% F1-Score trained in P1 (lemmas) and predicting the rest)
#MODEL_NAME = 'xlm-mlm-ende-1024' # this modell works even worse (40% F1-Score with heavy overfitting)

In [0]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['label'] = le.fit_transform(df.genre)

In [7]:
import ktrain
from ktrain import text
transformer = text.Transformer(MODEL_NAME, maxlen=512, classes=le.classes_.tolist())

using Keras version: 2.2.4-tf


In [8]:
df_train = df[df.period == 'P1']
df_test = df[df.period != 'P1']

train_data = transformer.preprocess_train(df_train.text, df_train.label)
test_data = transformer.preprocess_test(df_test.text, df_test.label)

preprocessing train...
language: en


preprocessing test...
language: en


In [0]:
model = transformer.get_classifier()

In [0]:
learner = ktrain.get_learner(model, train_data=train_data, val_data=test_data, batch_size=4)

In [0]:
#learner.lr_find(show_plot=True, max_epochs=2)

In [0]:
learner.autofit(2e-5, checkpoint_folder='/tmp/saved_weights') 

early_stopping automatically enabled at patience=5
reduce_on_plateau automatically enabled at patience=2


begin training using triangular learning rate policy with max lr of 2e-05...
Train for 29 steps, validate for 215 steps
Epoch 1/1024


In [0]:
learner.validate(class_names=transformer.get_classes())

In [0]:
! rm -rf /tmp/saved_weights